In [1]:
#execute this cell, then press 'Restart and clear cell outputs' and execute all cells except this
!pip install -U tensorflow==2.12.0rc0
!pip install -U numpy==1.18.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.8/585.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 28.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninst

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 16.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install contractions
!pip install vaderSentiment
!pip install sweetviz
!pip install textblob

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import contractions
import sweetviz as sv
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import time

#importing the Lemming function from nltk library
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt
from textblob import TextBlob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 26.3 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# Datasets

# Embeddings

In [4]:
path_text = "/content/gdrive/MyDrive/RedditDataset/ActiveLearning.csv"

df = pd.read_csv(path_text)
df = df.drop(['Unnamed: 0'], axis = 1)

print(df.columns)

df.head()

Index(['Related', 'original_text', 'full_text', 'embeddings'], dtype='object')


,Related,original_text,full_text,embeddings
0,0,NEED HELP With Router/Bridge Setup in Shop. So...,NEED HELP With Router/Bridge Setup in Shop. So...,[0.01063263 0.00574474 0.01323843 ... 0.022056...
1,0,Old laptop as a dedicated media streaming pc. ...,Old laptop as a dedicated media streaming pc. ...,[0.01515447 0.01004094 0.00107038 ... 0.028011...
2,0,Tenda Wireless Adapter giving poor speed. It's...,Tenda Wireless Adapter giving poor speed. It's...,[0.00670475 0.00331398 0.01671631 ... 0.010732...
3,0,Accidentally deleted Ethernet from Service opt...,Accidentally deleted Ethernet from Service opt...,[0.03719453 0.00892703 0.0035986 ... 0.007570...
4,0,"How to get an ""Elementor like"" design using Gu...","How to get an ""Elementor like"" design using Gu...",[-1.0716730e-05 1.2848090e-02 6.7767720e-03 ...


In [5]:
path_X = "/content/gdrive/MyDrive/RedditDataset/ActiveLearning_embeddings.csv"

X = np.loadtxt(path_X, delimiter=',')
X

array([[0.01063263, 0.00574474, 0.01323843, ..., 0.02205602, 0.01184695,
        0.03431676],
       [0.01515447, 0.01004094, 0.00107038, ..., 0.0280116 , 0.01383466,
        0.01741148],
       [0.00670475, 0.00331398, 0.01671631, ..., 0.01073266, 0.01530788,
        0.01552458],
       ...,
       [0.01530084, 0.00467629, 0.02155065, ..., 0.02255931, 0.00561598,
        0.01015869],
       [0.01813712, 0.01156727, 0.00410781, ..., 0.01426511, 0.02390902,
        0.03033417],
       [0.0055881 , 0.01479067, 0.00550449, ..., 0.01978885, 0.01427634,
        0.00826234]])

In [6]:
X.shape

(20000, 1024)

# Model


In [8]:
from tensorflow.keras.models import load_model

loaded_model = load_model('/content/gdrive/MyDrive/ExperimentalModels/2Layer_WB_tuned_FFNNmodel.h5')

In [9]:
predictions = loaded_model.predict(X)
predictions

625/625 [==============================] - 1s 2ms/step


array([[4.2829337e-04],
       [3.7710813e-03],
       [5.1209459e-04],
       ...,
       [9.0284320e-06],
       [1.6335340e-05],
       [1.9991453e-04]], dtype=float32)

In [10]:
predictions.shape

(20000, 1)

In [11]:
df = df.drop(columns=['embeddings'])
df

,Related,original_text,full_text
0,0,NEED HELP With Router/Bridge Setup in Shop. So...,NEED HELP With Router/Bridge Setup in Shop. So...
1,0,Old laptop as a dedicated media streaming pc. ...,Old laptop as a dedicated media streaming pc. ...
2,0,Tenda Wireless Adapter giving poor speed. It's...,Tenda Wireless Adapter giving poor speed. It's...
3,0,Accidentally deleted Ethernet from Service opt...,Accidentally deleted Ethernet from Service opt...
4,0,"How to get an ""Elementor like"" design using Gu...","How to get an ""Elementor like"" design using Gu..."
...,...,...,...
19995,0,Help! My PC is the only device with a slow int...,Help! My PC is the only device with a slow int...
19996,0,Is this website harmful?. I downloaded a book ...,Is this website harmful?. I downloaded a book ...
19997,0,"Is there anyway to fix ""CVE-2023-28531 affects...","Is there anyway to fix ""CVE-2023-28531 affects..."
19998,0,Microsoft SQL Server -> Excel export via front...,Microsoft SQL Server -> Excel export via front...


In [12]:
df['Prediction'] = predictions

for i in range(len(predictions)):
  if (predictions[i] < 0.5):
      predictions[i] = 0
  else:
      predictions[i] = 1

df['Related'] = predictions

df

,Related,original_text,full_text,Prediction
0,0.0,NEED HELP With Router/Bridge Setup in Shop. So...,NEED HELP With Router/Bridge Setup in Shop. So...,0.000428
1,0.0,Old laptop as a dedicated media streaming pc. ...,Old laptop as a dedicated media streaming pc. ...,0.003771
2,0.0,Tenda Wireless Adapter giving poor speed. It's...,Tenda Wireless Adapter giving poor speed. It's...,0.000512
3,0.0,Accidentally deleted Ethernet from Service opt...,Accidentally deleted Ethernet from Service opt...,0.000197
4,0.0,"How to get an ""Elementor like"" design using Gu...","How to get an ""Elementor like"" design using Gu...",0.000012
...,...,...,...,...
19995,0.0,Help! My PC is the only device with a slow int...,Help! My PC is the only device with a slow int...,0.021872
19996,1.0,Is this website harmful?. I downloaded a book ...,Is this website harmful?. I downloaded a book ...,0.999986
19997,0.0,"Is there anyway to fix ""CVE-2023-28531 affects...","Is there anyway to fix ""CVE-2023-28531 affects...",0.000009
19998,0.0,Microsoft SQL Server -> Excel export via front...,Microsoft SQL Server -> Excel export via front...,0.000016


In [42]:
relevant_posts = df[df['Related'] == 1]
print(relevant_posts.shape)

relevant_posts

(1889, 4)


,Related,original_text,full_text,Prediction
29,1.0,"Got this in the mail, invoice and receipt atta...","Got this in the mail, invoice and receipt atta...",0.633501
41,1.0,"How Can I Get Rid Wikilanda Virus(?). Hello, I...","How Can I Get Rid Wikilanda Virus(?). Hello, I...",0.990990
43,1.0,Best SysInternals Tools for Malware Analysis.,Best SysInternals Tools for Malware Analysis.,0.934719
50,1.0,"""The file can't be found"" dialogue box every t...","""The file can't be found"" dialogue box every t...",0.518368
63,1.0,Scary email - URGENT. Some months ago I got a ...,Scary email - URGENT. Some months ago I got a ...,0.996805
...,...,...,...,...
19938,1.0,Virus from allegedly safe site?. So I download...,Virus from allegedly safe site?. So I download...,0.999987
19970,1.0,The GitHub Actions Worm: Compromising GitHub R...,The GitHub Actions Worm: Compromising GitHub R...,0.998071
19975,1.0,Dark Tequila Ransomware Exposed: How Hackers c...,Dark Tequila Ransomware Exposed: How Hackers c...,0.998414
19976,1.0,Gaming. I was installing the atomic heart beta...,Gaming. I was installing the atomic heart beta...,0.684189


In [43]:
unrelevant_posts = df[df['Related'] == 0]
print(unrelevant_posts.shape)

unrelevant_posts

(18111, 4)


,Related,original_text,full_text,Prediction
0,0.0,NEED HELP With Router/Bridge Setup in Shop. So...,NEED HELP With Router/Bridge Setup in Shop. So...,0.000428
1,0.0,Old laptop as a dedicated media streaming pc. ...,Old laptop as a dedicated media streaming pc. ...,0.003771
2,0.0,Tenda Wireless Adapter giving poor speed. It's...,Tenda Wireless Adapter giving poor speed. It's...,0.000512
3,0.0,Accidentally deleted Ethernet from Service opt...,Accidentally deleted Ethernet from Service opt...,0.000197
4,0.0,"How to get an ""Elementor like"" design using Gu...","How to get an ""Elementor like"" design using Gu...",0.000012
...,...,...,...,...
19994,0.0,"Auto Repair.. Hey all, having a problem with b...","Auto Repair.. Hey all, having a problem with b...",0.014844
19995,0.0,Help! My PC is the only device with a slow int...,Help! My PC is the only device with a slow int...,0.021872
19997,0.0,"Is there anyway to fix ""CVE-2023-28531 affects...","Is there anyway to fix ""CVE-2023-28531 affects...",0.000009
19998,0.0,Microsoft SQL Server -> Excel export via front...,Microsoft SQL Server -> Excel export via front...,0.000016


In [44]:
for j in range(len(unrelevant_posts)):

  i = unrelevant_posts.index[j]

  value = unrelevant_posts.at[i, 'Prediction']

  unrelevant_posts.loc[i, 'Distance'] = abs( value - 0.5)

unrelevant_posts

<ipython-input-44-ef4dedf94a47>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unrelevant_posts.loc[i, 'Distance'] = abs( value - 0.5)


,Related,original_text,full_text,Prediction,Distance
0,0.0,NEED HELP With Router/Bridge Setup in Shop. So...,NEED HELP With Router/Bridge Setup in Shop. So...,0.000428,0.499572
1,0.0,Old laptop as a dedicated media streaming pc. ...,Old laptop as a dedicated media streaming pc. ...,0.003771,0.496229
2,0.0,Tenda Wireless Adapter giving poor speed. It's...,Tenda Wireless Adapter giving poor speed. It's...,0.000512,0.499488
3,0.0,Accidentally deleted Ethernet from Service opt...,Accidentally deleted Ethernet from Service opt...,0.000197,0.499803
4,0.0,"How to get an ""Elementor like"" design using Gu...","How to get an ""Elementor like"" design using Gu...",0.000012,0.499988
...,...,...,...,...,...
19994,0.0,"Auto Repair.. Hey all, having a problem with b...","Auto Repair.. Hey all, having a problem with b...",0.014844,0.485156
19995,0.0,Help! My PC is the only device with a slow int...,Help! My PC is the only device with a slow int...,0.021872,0.478128
19997,0.0,"Is there anyway to fix ""CVE-2023-28531 affects...","Is there anyway to fix ""CVE-2023-28531 affects...",0.000009,0.499991
19998,0.0,Microsoft SQL Server -> Excel export via front...,Microsoft SQL Server -> Excel export via front...,0.000016,0.499984


In [45]:
unrelevant_posts = unrelevant_posts.sort_values(by='Distance').head(1000)

unrelevant_posts = unrelevant_posts.drop(columns=['Distance'])

unrelevant_posts

,Related,original_text,full_text,Prediction
8226,0.0,I can't access virus and threat protection.. I...,I can't access virus and threat protection.. I...,0.499472
458,0.0,"In need of cybersecurity solutions, however I ...","In need of cybersecurity solutions, however I ...",0.499156
6597,0.0,Privacy error on chrome. I am having privacy e...,Privacy error on chrome. I am having privacy e...,0.498656
361,0.0,"USB Trojan Tutorial: How to Create, Deploy and...","USB Trojan Tutorial: How to Create, Deploy and...",0.497036
14159,0.0,How was this email compromised?. We had a user...,How was this email compromised?. We had a user...,0.496264
...,...,...,...,...
8336,0.0,Anonymous DDoS Hacking.,Anonymous DDoS Hacking.,0.098777
18082,0.0,Best tools to make server room refresh as easy...,Best tools to make server room refresh as easy...,0.098481
18142,0.0,Couple decide to create shortcut to make it ea...,Couple decide to create shortcut to make it ea...,0.098432
5892,0.0,Job offer seems fishy to me...scam?. So I appl...,Job offer seems fishy to me...scam?. So I appl...,0.098428


In [46]:
for j in range(len(relevant_posts)):

  i = relevant_posts.index[j]

  value = relevant_posts.at[i, 'Prediction']

  relevant_posts.loc[i, 'Distance'] = abs( value - 0.5)

relevant_posts

<ipython-input-46-bdf649c5751a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_posts.loc[i, 'Distance'] = abs( value - 0.5)


,Related,original_text,full_text,Prediction,Distance
29,1.0,"Got this in the mail, invoice and receipt atta...","Got this in the mail, invoice and receipt atta...",0.633501,0.133501
41,1.0,"How Can I Get Rid Wikilanda Virus(?). Hello, I...","How Can I Get Rid Wikilanda Virus(?). Hello, I...",0.990990,0.490990
43,1.0,Best SysInternals Tools for Malware Analysis.,Best SysInternals Tools for Malware Analysis.,0.934719,0.434719
50,1.0,"""The file can't be found"" dialogue box every t...","""The file can't be found"" dialogue box every t...",0.518368,0.018368
63,1.0,Scary email - URGENT. Some months ago I got a ...,Scary email - URGENT. Some months ago I got a ...,0.996805,0.496805
...,...,...,...,...,...
19938,1.0,Virus from allegedly safe site?. So I download...,Virus from allegedly safe site?. So I download...,0.999987,0.499987
19970,1.0,The GitHub Actions Worm: Compromising GitHub R...,The GitHub Actions Worm: Compromising GitHub R...,0.998071,0.498071
19975,1.0,Dark Tequila Ransomware Exposed: How Hackers c...,Dark Tequila Ransomware Exposed: How Hackers c...,0.998414,0.498414
19976,1.0,Gaming. I was installing the atomic heart beta...,Gaming. I was installing the atomic heart beta...,0.684189,0.184189


In [48]:
relevant_posts = relevant_posts.sort_values(by='Distance').head(1000)

relevant_posts

,Related,original_text,full_text,Prediction,Distance
15175,1.0,I recently received this suspicious email/atta...,I recently received this suspicious email/atta...,0.500938,0.000938
16352,1.0,win server 2008 r2 - Console Root Cert folder ...,win server 2008 r2 - Console Root Cert folder ...,0.501249,0.001249
4606,1.0,anyone know a subdomain dns site. Im looking f...,anyone know a subdomain dns site. Im looking f...,0.501710,0.001710
2494,1.0,Wikipedia Redirecting To Malware (And Still Is...,Wikipedia Redirecting To Malware (And Still Is...,0.501834,0.001834
11453,1.0,I clicked a tracking link and I was hoping som...,I clicked a tracking link and I was hoping som...,0.502153,0.002153
...,...,...,...,...,...
4398,1.0,Remcos RAT - Malware Analysis Lab.,Remcos RAT - Malware Analysis Lab.,0.988074,0.488074
8700,1.0,I think someone is seeing all my online activi...,I think someone is seeing all my online activi...,0.988116,0.488116
10588,1.0,S1 unquarantine file/ download threat. Have a ...,S1 unquarantine file/ download threat. Have a ...,0.988230,0.488230
19772,1.0,"WhatsApp hacked. Hi all,\n\nI am pretty sure m...","WhatsApp hacked. Hi all,\n\nI am pretty sure m...",0.988411,0.488411


In [49]:

relevant_posts = relevant_posts.drop(columns=['Distance'])

relevant_posts


,Related,original_text,full_text,Prediction
15175,1.0,I recently received this suspicious email/atta...,I recently received this suspicious email/atta...,0.500938
16352,1.0,win server 2008 r2 - Console Root Cert folder ...,win server 2008 r2 - Console Root Cert folder ...,0.501249
4606,1.0,anyone know a subdomain dns site. Im looking f...,anyone know a subdomain dns site. Im looking f...,0.501710
2494,1.0,Wikipedia Redirecting To Malware (And Still Is...,Wikipedia Redirecting To Malware (And Still Is...,0.501834
11453,1.0,I clicked a tracking link and I was hoping som...,I clicked a tracking link and I was hoping som...,0.502153
...,...,...,...,...
4398,1.0,Remcos RAT - Malware Analysis Lab.,Remcos RAT - Malware Analysis Lab.,0.988074
8700,1.0,I think someone is seeing all my online activi...,I think someone is seeing all my online activi...,0.988116
10588,1.0,S1 unquarantine file/ download threat. Have a ...,S1 unquarantine file/ download threat. Have a ...,0.988230
19772,1.0,"WhatsApp hacked. Hi all,\n\nI am pretty sure m...","WhatsApp hacked. Hi all,\n\nI am pretty sure m...",0.988411


In [51]:
activeLearningPosts = pd.concat([relevant_posts, unrelevant_posts])
activeLearningPosts = activeLearningPosts.sample(frac=1).reset_index(drop=True)
activeLearningPosts

,Related,original_text,full_text,Prediction
0,0.0,"Accidentally clicked on a link in job email, m...","Accidentally clicked on a link in job email, m...",0.310937
1,0.0,Vulnerabilities in Supermicro BMCs could allow...,Vulnerabilities in Supermicro BMCs could allow...,0.161743
2,1.0,Can't create an email as it both exists and do...,Can't create an email as it both exists and do...,0.559433
3,1.0,Kohberger’s defense team runs parallel investi...,Kohberger’s defense team runs parallel investi...,0.555590
4,1.0,Microsoft Ticking Timebombs - March 2023 Editi...,Microsoft Ticking Timebombs - March 2023 Editi...,0.693611
...,...,...,...,...
1995,1.0,Hi Guys Am I safe?. So I was going through dis...,Hi Guys Am I safe?. So I was going through dis...,0.917349
1996,0.0,RNC wonen dieven Christel koning en Remco Aart...,RNC wonen dieven Christel koning en Remco Aart...,0.384984
1997,1.0,Failed to create process. I changed the name o...,Failed to create process. I changed the name o...,0.665957
1998,1.0,Trump allies orchestrate disinformation plot t...,Trump allies orchestrate disinformation plot t...,0.595025


In [52]:
activeLearningPosts['Checked'] = 0
activeLearningPosts['Manual Evaluation'] = 0

activeLearningPosts = activeLearningPosts[['original_text', 'full_text', 'Prediction', 'Related', 'Checked', 'Manual Evaluation']]
activeLearningPosts

,original_text,full_text,Prediction,Related,Checked,Manual Evaluation
0,"Accidentally clicked on a link in job email, m...","Accidentally clicked on a link in job email, m...",0.310937,0.0,0,0
1,Vulnerabilities in Supermicro BMCs could allow...,Vulnerabilities in Supermicro BMCs could allow...,0.161743,0.0,0,0
2,Can't create an email as it both exists and do...,Can't create an email as it both exists and do...,0.559433,1.0,0,0
3,Kohberger’s defense team runs parallel investi...,Kohberger’s defense team runs parallel investi...,0.555590,1.0,0,0
4,Microsoft Ticking Timebombs - March 2023 Editi...,Microsoft Ticking Timebombs - March 2023 Editi...,0.693611,1.0,0,0
...,...,...,...,...,...,...
1995,Hi Guys Am I safe?. So I was going through dis...,Hi Guys Am I safe?. So I was going through dis...,0.917349,1.0,0,0
1996,RNC wonen dieven Christel koning en Remco Aart...,RNC wonen dieven Christel koning en Remco Aart...,0.384984,0.0,0,0
1997,Failed to create process. I changed the name o...,Failed to create process. I changed the name o...,0.665957,1.0,0,0
1998,Trump allies orchestrate disinformation plot t...,Trump allies orchestrate disinformation plot t...,0.595025,1.0,0,0


In [53]:
path_text = "/content/gdrive/MyDrive/RedditPosts/ActiveLearningPosts.csv"


In [54]:
# Save df to CSV

activeLearningPosts.to_csv(path_text)